<a href="https://colab.research.google.com/github/JasperJZhou/github-slideshow/blob/master/DIGHUM100PROJ_JasperZhou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**0) IMPORT STATEMENTS**

Importing Data Analysis Tools and mounting the Google Drive

In [1]:
#Mounting Google Drive:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Jasper Zhou/' #Change this to your root path
drive.mount("/content/gdrive", force_remount=True) 


Mounted at /content/gdrive
Mounted at /content/gdrive


In [18]:
#IMPORT STATEMENTS:

#General data analysis tools 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

#NLTK Tools
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


import nltk
import nltk.tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import re #regular expression

#Model Building Tools:
from sklearn.model_selection import train_test_split

#Classification Algorithms
from sklearn.naive_bayes import MultinomialNB #Naive Bayes
from sklearn import svm #Support Vector Machine
from sklearn import linear_model #Logistic Regression

##**I) LOADING THE DATASET:**
Create a DataFrame using pandas with our downloaded data (in folder marked data)


In [3]:
true_df = pd.read_csv(root_path + '/data/True.csv') #DataFrame with True headlines (Is Not Fake News)
fake_df = pd.read_csv(root_path + '/data/Fake.csv') #DataFrame with Fake headlines (Is Fake News)

In [4]:
#Label our data (1 for fake news, 0 for true news)
true_df["Is_Fake"] = np.zeros(true_df.shape[0])
fake_df["Is_Fake"] = np.ones(fake_df.shape[0])

In [5]:
#Combine DataFrames into a CombinedFrame:
combined_df = true_df.append(fake_df)
combined_df.head()

,title,text,subject,date,Is_Fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0.0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0.0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0.0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0.0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0.0


##**II)INTRODUCTION**

#Project Statement
The goal of this project is two-fold:

A)Can we analyze how the rhetoric around fake news has changed throughout time?

B)Can we create an effective model to classify fake and real news?

#Background

To start, a definition of fake news is needed. While many definitions exist, a good, succinct one comes from the University of Michigan's library, which defines fake news "as those news stories that are false: the entire story itself is fabricated, with no verifiable facts, sources or quotes" (Source: [University of Michigan](https://guides.lib.umich.edu/fakenews)).

Fake news has recently taken center stage in the public conciousness, popularized by tweets by Donald Trump in the run up to the 2016 presidential election. However, this project will not focus on the media relations of Donald Trump. Instead, this project will focus on the nature of fake news itself in terms of content.

Fake news is not a recent or unique issue, although it has been exacerbated by the ease and speed in which information could be spread. Since the 1890s, fake news has plagued the profession of journalism. Dubbed "yellow journalism", newspaper publishers would run sensationlistic news articles. However, fake news was eventually replaced by objective journalism, until recently due to the rise of the internet. (Source: [UCSB](https://www.cits.ucsb.edu/fake-news/brief-history)).

Unfortunately, the future seems bleak for traditional, objective news outlets. In 2016, Politico reported the findings of the Pew Research Center's "State of the Media 2016", noted that "[a]dvertising revenue is down; staffs continue to get cut; the number of newspapers has declined by 100 since 2004". (Source: Politico). As a result, we are moving towards a new world "where the majority of the population does not reply on professionally reported news sources" (Source: [Politico](https://www.politico.com/magazine/story/2016/12/fake-news-history-long-violent-214535)).

In terms of impacts fake news has on society, University of Michigan lists the following as possibility:

**A) Anti-intellectualism:** We have seen during this past year due to COVID-19, fake social media stories have run amok spanning from fake treatments to vaccine scares. Often, these issues stem from a distrust of domain knowledge experts. For example, according to a letter written to NCBI, several incidents exemplified this. In a particularly greivious instance in Mexico:


> [T]here have been reports of health personnel having hot coffee or bleach thrown at them on the streets, as well as attempts to burn their homes, and even beatings of nurses and doctors; in some towns, the habitants have even arranged to burn COVID-19-designated hospitals. (Source: [NCBI](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7722992/))

**B)Antiscience:** Similar to anti-intellectualism, there has been a startling rise in anti-scientific theories. For example, the Flat Earth movement has demonstrated the power of social media in overturning what used to be a universally accepted truth. In an interview with Scientific American, Michael Marshall noted that the Flat Earth movement's rise is closely correlated with the technology we have available. In his interview, a contributing factor was Youtube's video recommendation algorithm. It became very easy for individuals, who were only curious, to quickly get recommended similar videos, reinforcing their beliefs. (Source: [Scientific American](https://www.scientificamerican.com/podcast/episode/flat-earthers-what-they-believe-and-why/))

**C)Widespread Mistrust:** In 2016, the BBC interviewed Robert Proctor who had been studying the rhetoric surrounding tobacco advertisements. The goal of these tobacco advertisements were not necessarily to discredit scientific literature and studies, but to create enough doubt surrounding the scientific community at large. Prior to 2016, similar strategies were used surrounding Barack Obama's birth certificate. While Obama was undoubtedly American, the rumor successfully created enough doubt around Obama's birth place to create an unncessarily harmful and dangerous scandal. (Source: [BBC](https://www.bbc.com/future/article/20160105-the-man-who-studies-the-spread-of-ignorance))



##**III) EDA (Exploratory Data Analysis)**




#i)Intial Pre-processing:
First, we must pre-process the text to create text that is easily machine-readable. To do so, we undergo the following steps:


1. Lowercase all text
2. Tokenize the sentence into individual words
3. Remove the stop words

From this, we can do some initial, basic data analysis and examine the differences between real and fake news.


In [19]:
#TESTER: Create small subset of data to test/create our pipeline on.
pre_process_df_test = combined_df.sample(50, replace = False)

#Lowercase all titles and text
pre_process_df_test["title"] = pre_process_df_test["title"].str.lower()
pre_process_df_test["text"] = pre_process_df_test["text"].str.lower()

#tokenize words into individual words
pre_process_df_test["tokenized_title"] = pre_process_df_test["title"].apply(word_tokenize)


LookupError: ignored

##**IV)SENTIMENT ANALYSIS OF FAKE/REAL NEWS**

##**V)PREDICTING FAKE/REAL NEWS**